In [12]:
import pandas as pd
import matplotlib.pylab as plt
import string                            #导入字符串模块

# 数据预处理

In [13]:
tab1 = "./hair_dryer.tsv"

df = pd.read_csv(tab1, sep='\t', header=0)

## 删除无效信息

In [16]:
df = df[~(df['vine'].str.contains("N") & df['verified_purchase'].str.contains("N"))]

In [17]:
df = df[(df['total_votes'] > 0) & (df['helpful_votes'] > 0)]

In [27]:
df = df.groupby('product_parent').filter(lambda x: len(x) > 1)

In [29]:
print(len(df))

2775


# 整体文本分析

In [30]:
pattern = r"\&\#[0-9]+\;"

df["preprocessed"] = df["review_body"].str.replace(pat=pattern, repl="", regex=True)

In [31]:
reviews = ''
for review in df['review_body']:
    reviews = reviews + ' ' + review
hist = {}                                 #创建一个空字典，放词频与单词，无序排列
data = []                                 #创建一个空列表，放词频与单词，有序：从多到少
content = reviews.replace('-',' ')       #连字符—用空格代替
words = content.split()                   #字符串按空格分割--分词

## 删除介词

In [32]:
excludes={"not","if","about","really","too","br","has","very", "so","on","at","when", "was","one","had", "it's","than","would","the","and","of","you","a","with","but","as","be","in","or","are", "i", "it", "to", "hair","this", "is", "my", "dryer", "for", "that", "have"}
for word in excludes:
    if word in hist:
        del(hist[word])